# Rejilla de datos diarios de P y T de AEMET discretización 5 km
https://www.aemet.es/es/serviciosclimaticos/cambio_climat/datos_diarios?w=2<br>


In [4]:
import pandas as pd
import numpy as np
import time

## Rejillas
Las 3 rejillas de pdia, tmindia y tmaxdia son iguales 

In [43]:
fgrids = ('.\\pcp_1951a2021\\maestro_red_hr_SPAIN.txt',
          '.\\tmin_1951a2021\maestro_red_hr_tmin.txt',
          '.\\tmax_1951a2021\maestro_red_hr_tmax.txt')

for f in fgrids:
    df = pd.read_csv(f, delim_whitespace=True, header=None, names=('id', 'lon', 'lat', 'z'))
    print(df.head(3))
    print('...')

   id    lon     lat      z
0   1 -2.948  35.307  102.0
1   2 -5.323  35.898   58.0
2   3 -5.630  36.049  124.0
...
   id    lon     lat      z
0   1 -2.948  35.307  102.0
1   2 -5.323  35.898   58.0
2   3 -5.630  36.049  124.0
...
   id    lon     lat      z
0   1 -2.948  35.307  102.0
1   2 -5.323  35.898   58.0
2   3 -5.630  36.049  124.0
...


In [42]:
fo = 'maestro_red_hr_SPAIN.csv'
df.to_csv(fo, index=False)

## Read txt file of Pd provided by AEMET
### Read Daily P

In [31]:
f = '.\\pcp_1951a2021\\pcp_red_SPAIN_1951-2021.txt'
df = pd.read_csv(f, delim_whitespace=True, header=None, nrows=4, parse_dates=[0])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16147,16148,16149,16150,16151,16152,16153,16154,16155,16156
0,1951-01-01,0.37,8.05,17.80,17.89,16.11,15.42,15.96,16.79,18.46,...,24.89,23.23,21.89,20.60,25.13,24.03,23.06,23.01,22.63,19.48
1,1951-01-02,4.29,18.46,8.65,9.47,10.30,6.03,5.98,6.18,6.86,...,19.31,19.80,20.05,20.13,14.14,15.59,16.81,17.68,18.37,19.40
2,1951-01-03,0.00,1.55,0.00,0.93,1.65,0.00,0.00,0.00,0.89,...,3.17,2.83,2.55,2.27,0.98,1.04,1.13,1.47,1.68,1.13
3,1951-01-04,0.00,0.32,2.01,1.99,1.92,2.02,2.16,2.27,2.30,...,0.25,0.26,0.25,0.24,0.23,0.30,0.33,0.31,0.30,0.32


In [32]:
cols = [c for c in df]
float32_cols = {c: np.float32 for c in cols[1:]}
len(float_cols)

16156

In [33]:
t0 = time.time()
df = pd.read_csv(f, delim_whitespace=True, header=None, parse_dates=[0], engine='c', dtype=float32_cols)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

7.40 min


### Save daily P df to parquet / feather file format
Column's names must be strings

In [34]:
%%timeit
df.columns = df.columns.astype(str)

312 µs ± 35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [37]:
fo = '.\\pcp_1951a2021\\pcp_red_SPAIN_1951-202.parquet'

t0 = time.time()
df.to_parquet(fo, engine = 'fastparquet')
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.41 min


In [38]:
fo = '.\\pcp_1951a2021\\pcp_red_SPAIN_1951-202.feather'

t0 = time.time()
df.to_feather(fo)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.12 min


### Test reading some columns

In [39]:
f = '.\\pcp_1951a2021\\pcp_red_SPAIN_1951-202.parquet'
columns_to_include = ['0', '1', '4000']

t0 = time.time()
df = pd.read_parquet(f, columns=columns_to_include)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.01 min


In [40]:
f = '.\\pcp_1951a2021\\pcp_red_SPAIN_1951-202.feather'
columns_to_include = ['0', '1', '4000']

t0 = time.time()
df = pd.read_feather(f, columns=columns_to_include)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.00 min


## Read geo file of daily Tmin provided by AEMET and save to parquet format
### Read Daily Tmin

In [19]:
t0 = time.time()
f = '.\\tmin_1951a2021_txt\\tmin_red_SPAIN_1951-2021.geo'

df = pd.read_csv(f, delim_whitespace=True, header=None, parse_dates=[0], engine='c', dtype=float32_cols, nrows=5)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.02 min


In [20]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16147,16148,16149,16150,16151,16152,16153,16154,16155,16156
0,1951-01-01,5.62,8.71,7.75,7.59,8.12,8.28,7.46,7.25,6.87,...,6.09,6.31,6.56,6.84,6.81,7.15,7.50,7.15,7.18,7.93
1,1951-01-02,8.35,10.82,10.01,9.79,10.32,10.70,9.85,9.57,9.10,...,4.32,4.48,4.66,4.91,5.10,5.38,5.70,5.32,5.32,6.01
2,1951-01-03,7.55,8.80,7.67,7.52,8.08,8.21,7.39,7.14,6.77,...,3.80,4.02,4.24,4.52,4.52,4.86,5.21,4.86,4.89,5.64
3,1951-01-04,6.57,8.47,7.32,7.22,7.79,7.72,6.94,6.75,6.50,...,5.25,5.47,5.69,5.94,6.22,6.60,6.91,6.50,6.47,7.19
4,1951-01-05,6.72,9.87,8.84,8.72,9.22,9.34,8.53,8.28,8.00,...,8.22,8.47,8.69,8.97,9.12,9.50,9.87,9.47,9.47,10.25


In [21]:
%%timeit
df.columns = df.columns.astype(str)

317 µs ± 31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
f = '.\\tmin_1951a2021_txt\\tmin_red_SPAIN_1951-2021.geo'

t0 = time.time()
df = pd.read_csv(f, delim_whitespace=True, header=None, parse_dates=[0], engine='c', dtype=float32_cols)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

7.68 min


### Save to parquet

In [15]:
fo = '.\\tmin_1951a2021_txt\\tmin_red_SPAIN_1951-2021.parquet'

t0 = time.time()
df.to_parquet(fo, engine = 'fastparquet')
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')


0.39 min


### Read parquet selected columns

In [24]:
f = '.\\tmin_1951a2021_txt\\tmin_red_SPAIN_1951-2021.parquet'
columns_to_include = ['0', '1', '4000']

t0 = time.time()
df = pd.read_parquet(f, columns=columns_to_include)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.01 min


In [26]:
df.head()

,0,1,4000
0,1951-01-01,5.62,1.59
1,1951-01-02,8.35,2.82
2,1951-01-03,7.55,1.17
3,1951-01-04,6.57,-0.56
4,1951-01-05,6.72,-0.00


## Read geo file of daily Tmax provided by AEMET and save to parquet format
### Read Daily Tmax

In [27]:
f = '.\\tmax_1951a2021\\tmax_red_SPAIN_1951-2021.geo'

t0 = time.time()
df = pd.read_csv(f, delim_whitespace=True, header=None, parse_dates=[0], engine='c', dtype=float32_cols)
df.columns = df.columns.astype(str)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

7.38 min


### Save Tmax df to parquet

In [28]:
fo = '.\\tmax_1951a2021\\tmax_red_SPAIN_1951-2021.parquet'

t0 = time.time()
df.to_parquet(fo, engine = 'fastparquet')
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.44 min


### Test reading some columns

In [29]:
f = '.\\tmax_1951a2021\\tmax_red_SPAIN_1951-2021.parquet'
columns_to_include = ['0', '1', '4000']

t0 = time.time()
df = pd.read_parquet(f, columns=columns_to_include)
t1 = (time.time() - t0) / 60.
print(f'{t1:0.2f} min')

0.01 min


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25933 entries, 0 to 25932
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       25933 non-null  datetime64[ns]
 1   1       25933 non-null  float32       
 2   4000    25933 non-null  float32       
dtypes: datetime64[ns](1), float32(2)
memory usage: 405.3 KB
